In [ ]:
import os
import pickle
import numpy as np
import cv2 as cv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from metrics import compute_distance
from histograms import compute_descriptor 
import evaluation
import utils

%matplotlib inline

In [ ]:
dataset_dir = "{}/BBDD".format(os.getcwd()) # dataset path
queryset_dir = "{}/qsd1_w1".format(os.getcwd()) # query set path
# testset_dir = "{}/qst1_w1".format(os.getcwd()) # test set path
database_dir = "{}/database".format(os.getcwd()) # database path
result_dir = "{}/results".format(os.getcwd()) ## result path

"""
Available descriptors:
    - gray_histogram,
    - hsv_1d_histogram,
    - rgb_1d_histogram
    - ycrcb_1d_histogram,
    - lab_1d_histogram
    - hsv_3d_histogram,
    - rgb_3d_histogram,
    - ycrcb_3d_histogram,
    - lab_3d_histogram

Available metrics:
    - cosine,
    - euclidean,
    - hellinger_kernel,
    - chi_square,
    - correlation,
    - intersection
"""
metric = 'euclidean'
descriptor = 'ycrcb_3d_histogram' 

k = 10

results = []
# ground_truth = utils.load_pickle('gt_corresps_2.pkl')
ground_truth = [[276], [272], [155], [22], [157], [23], [250], [110], [215], [280], [192], [184], [77], [258], [100], [155], [92], [225], [130], [251], [120], [203], [91], [186], [52], [262], [227], [53], [142], [32]]
histograms = utils.load_database(descriptor, dataset_dir, database_dir)

query_set = [fname for fname in sorted(os.listdir(queryset_dir)) if fname.endswith(".jpg")]
for idx, query in enumerate(query_set):
    print("Query {}: {}, ground-truth: {}".format(idx, query, str(ground_truth[idx][0]).zfill(5)))
       
    query_hist = compute_descriptor(descriptor, os.path.join(queryset_dir, query))
    
    k_nearest = utils.k_nearest_search(histograms, query_hist, ground_truth[idx][0], metric, k)
    
    print("{}-most similar images:".format(k))
    result = []
    for i, image in enumerate(k_nearest):
        print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
        result.append(image[-1])
        if i == k - 1:
            results.append(result)   
    print("==================================")
print(results)    
utils.save_pickle(result_dir + '/result_' + descriptor + '_top' + str(k) + '.pkl', results)
print(evaluation.mapk(ground_truth, results, k))

In [ ]:
"""
Compute the ratio of correct prediction to see how well it works
"""
count = 0
for i, (a,b) in enumerate(zip(ground_truth, results)):
    if a[0] in b:
        print("OK".format(i))
        count += 1
    else:
        print("Not OK".format(i))
print("{}/30". format(count))    